<div style="text-align: center;">
    <h1 style="color: #FF6347;">Self-Guided Lab: Retrieval-Augmented Generation (RAGs)</h1>
</div>

<div style="text-align: center;">
    <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZ3FsdzRveTBrenMxM3VnbDMwaTJxN2NnZm50aGFibXk1NzNnY2Q0MCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/LR5ZBwZHv02lmpVoEU/giphy.gif" alt="NLP Gif" style="width: 300px; height: 150px; object-fit: cover; object-position: center;">
</div>

<h1 style="color: #FF6347;">Data Storage & Retrieval</h1>


<h2 style="color: #FF8C00;">PyPDFLoader</h2>

`PyPDFLoader` is a lightweight Python library designed to streamline the process of loading and parsing PDF documents for text processing tasks. It is particularly useful in Retrieval-Augmented Generation workflows where text extraction from PDFs is required.

- **What Does PyPDFLoader Do?**
  - Extracts text from PDF files, retaining formatting and layout.
  - Simplifies the preprocessing of document-based datasets.
  - Supports efficient and scalable loading of large PDF collections.

- **Key Features:**
  - Compatible with popular NLP libraries and frameworks.
  - Handles multi-page PDFs and embedded images (e.g., OCR-compatible setups).
  - Provides flexible configurations for structured text extraction.

- **Use Cases:**
  - Preparing PDF documents for retrieval-based systems in RAGs.
  - Automating the text extraction pipeline for document analysis.
  - Creating datasets from academic papers, technical manuals, and reports.


In [1]:
%pip install langchain langchain_community pypdf
%pip install termcolor langchain_openai langchain-huggingface sentence-transformers chromadb langchain_chroma tiktoken openai python-dotenv


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings('ignore')


In [3]:
import os 
import csv
from datetime import datetime

# function to append results to csv
def log_replies(embedder,chunk_size,user_prompt,total_input,response,params,file='results.csv'):

    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    fields=[timestamp, 
            embedder,
            chunk_size,
            user_prompt,
            total_input,
            response,
            params
            ]
    headers = ['time','embedder', 'chunk-size', 'userprompt','total-input','response','params']
    
    
    file_exists = os.path.exists(file)

    with open(file, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(headers)
        writer.writerow(fields)

<h3 style="color: #FF8C00;">Loading the Documents</h3>

In [4]:
# File path for the document

file_path = "ai-for-everyone.pdf"

<h3 style="color: #FF8C00;">Documents into pages</h3>

The `PyPDFLoader` library allows efficient loading and splitting of PDF documents into smaller, manageable parts for NLP tasks.

This functionality is particularly useful in workflows requiring granular text processing, such as Retrieval-Augmented Generation (RAG).


In [5]:
# Load and split the document
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)

297

<h3 style="color: #FF8C00;">Pages into Chunks</h3>


####  RecursiveCharacterTextSplitter in LangChain

The `RecursiveCharacterTextSplitter` is the **recommended splitter** in LangChain when you want to break down long documents into smaller, semantically meaningful chunks — especially useful in **RAG pipelines**, where clean context chunks lead to better LLM responses.

####  Parameters

| Parameter       | Description                                                                 |
|-----------------|-----------------------------------------------------------------------------|
| `chunk_size`    | The **maximum number of characters** allowed in a chunk (e.g., `1000`).     |
| `chunk_overlap` | The number of **overlapping characters** between consecutive chunks (e.g., `200`). This helps preserve context continuity. |

####  How it works
`RecursiveCharacterTextSplitter` attempts to split the text **intelligently**, trying the following separators in order:
1. Paragraphs (`"\n\n"`)
2. Lines (`"\n"`)
3. Sentences or words (`" "`)
4. Individual characters (as a last resort)

This makes it ideal for handling **natural language documents**, such as PDFs, articles, or long reports, without breaking sentences or paragraphs in awkward ways.



In [6]:
chunk_size = 1500
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(pages)

len(chunks)

764

In [7]:
type(chunks)

list

In [ ]:
delist=[]
for i in range(len(chunks)):
    if len(chunks[i].page_content) < 60:
        delist.append(i)
        
len(delist)
delist.sort(reverse=True)

for i in delist:
    print(i)
    del(chunks[i])

In [9]:
delist

[62, 225, 463]

463
225
62


In [12]:
chunks

[Document(metadata={'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2021-09-01T15:24:38+05:30', 'author': 'Pieter Verdegem', 'moddate': '2021-09-14T17:31:25-04:00', 'title': 'AI for Everyone?: Critical Perspectives', 'trapped': '/False', 'source': 'ai-for-everyone.pdf', 'total_pages': 310, 'page': 0, 'page_label': ''}, page_content='AI FOR EVERYONE?  CRITICAL PERSPECTIVESPIETER VERDEGEM /parenleft.caseED ./parenright.case\n uwestminsterpress.co.uk\nAI FOR EVERYONE?\nW\ne are entering a new era of technological determinism and \nsolutionism in which governments and business actors are \nseeking data-driven change, assuming that Artiﬁ  cial Intelligence \nis now inevitable and ubiquitous. But we have not even started asking the \nright questions, let alone developed an understanding of the consequences. \nUrgently needed is debate that asks and answers fundamental questions \nabout power.\nThis book brings together critical interrogati

####  Alternative: CharacterTextSplitter

`CharacterTextSplitter` is a simpler splitter that breaks text into chunks based **purely on character count**, without trying to preserve any natural language structure.

##### Example:
```python
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
````

This method is faster and more predictable but may split text in the middle of a sentence or paragraph, which can hurt performance in downstream tasks like retrieval or QA.

---

#### Comparison Table

| Feature                        | RecursiveCharacterTextSplitter | CharacterTextSplitter     |
| ------------------------------ | ------------------------------ | ------------------------- |
| Structure-aware splitting      |  Yes                          |  No                      |
| Preserves sentence/paragraphs  |  Yes                          |  No                      |
| Risk of splitting mid-sentence |  Minimal                     |  High                   |
| Ideal for RAG/document QA      |  Highly recommended           |  Only if structured text |
| Performance speed              |  Slightly slower             |  Faster                  |

---

#### Recommendation

Use `RecursiveCharacterTextSplitter` for most real-world document processing tasks, especially when building RAG pipelines or working with structured natural language content like PDFs or articles.

## Best Practices for Choosing Chunk Size in RAG

### Best Practices for Chunk Size in RAG

| Factor                      | Recommendation                                                                                                                                                                                          |
| ---------------------------| ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **LLM context limit**       | Choose a chunk size that lets you retrieve multiple chunks **without exceeding the model’s token limit**. For example, GPT-4o supports 128k tokens, but with GPT-3.5 (16k) or GPT-4 (32k), keep it modest. |
| **Chunk size (in characters)** | Typically: **500–1,000 characters** per chunk → ~75–200 tokens. This fits well for retrieval + prompt without context overflow.                                                                           |
| **Chunk size (in tokens)**  | If using token-based splitter (e.g. `TokenTextSplitter`): aim for **100–300 tokens** per chunk.                                                                                                            |
| **Chunk overlap**           | Use **overlap of 10–30%** (e.g., 100–300 characters or ~50 tokens) to preserve context across chunk boundaries and avoid cutting off important ideas mid-sentence.                                        |
| **Document structure**      | Use **`RecursiveCharacterTextSplitter`** to preserve semantic boundaries (paragraphs, sentences) instead of arbitrary cuts.                                                                                |
| **Task type**               | For **question answering**, smaller chunks (~500–800 chars) reduce noise.<br>For **summarization**, slightly larger chunks (~1000–1500) are OK.                                                          |
| **Embedding model**         | Some models (e.g., `text-embedding-3-large`) can handle long input. But still, smaller chunks give **finer-grained retrieval**, which improves relevance.                                                  |
| **Query type**              | If users ask **very specific questions**, small focused chunks are better. For broader queries, bigger chunks might help.                                                                                  |


### Rule of Thumb

| Use Case                 | Chunk Size      | Overlap |
| ------------------------| --------------- | ------- |
| Factual Q&A              | 500–800 chars   | 100–200 |
| Summarization            | 1000–1500 chars | 200–300 |
| Technical documents      | 400–700 chars   | 100–200 |
| Long reports/books       | 800–1200 chars  | 200–300 |
| Small LLMs (≤16k tokens) | ≤800 chars      | 100–200 |


### Avoid

- Chunks >2000 characters: risks context overflow.
- No overlap: may lose key information between chunks.



<h2 style="color: #FF8C00;">Embeddings</h2>

Embeddings transform text into dense vector representations, capturing semantic meaning and contextual relationships. They are essential for efficient document retrieval and similarity analysis.

- **What are OpenAI Embeddings?**
  - Pre-trained embeddings like `text-embedding-3-large` generate high-quality vector representations for text.
  - Encapsulate semantic relationships in the text, enabling robust NLP applications.

- **Key Features of `text-embedding-3-large`:**
  - Large-scale embedding model optimized for accuracy and versatility.
  - Handles diverse NLP tasks, including retrieval, classification, and clustering.
  - Ideal for applications with high-performance requirements.

- **Benefits:**
  - Reduces the need for extensive custom training.
  - Provides state-of-the-art performance in retrieval-augmented systems.
  - Compatible with RAGs to create powerful context-aware models.


In [13]:
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

In [14]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List

class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return [self.model.encode(d).tolist() for d in documents]

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0].tolist()


embedder="all-MiniLM-L6-v2"
embeddingsST = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<h2 style="color: #FF8C00;">ChromaDB</h2>

ChromaDB is a versatile vector database designed for efficiently storing and retrieving embeddings. It integrates seamlessly with embedding models to enable high-performance similarity search and context-based retrieval.

### Workflow Overview:
- **Step 1:** Generate embeddings using a pre-trained model (e.g., OpenAI's `text-embedding-3-large`).
- **Step 2:** Store the embeddings in ChromaDB for efficient retrieval and similarity calculations.
- **Step 3:** Use the stored embeddings to perform searches, matching, or context-based retrieval.

### Key Features of ChromaDB:
- **Scalability:** Handles large-scale datasets with optimized indexing and search capabilities.
- **Speed:** Provides fast and accurate retrieval of embeddings for real-time applications.
- **Integration:** Supports integration with popular frameworks and libraries for embedding generation.

In [19]:
from langchain.vectorstores import Chroma


In [20]:
db = Chroma.from_documents(chunks, embeddingsST, persist_directory="./chroma_db_AI")
print("ChromaDB created with document embeddings.")

ChromaDB created with document embeddings.


<h1 style="color: #FF6347;">Retrieving Documents</h1>


### Exercice1: Write a user question that someone might ask about your book’s topic or content.

In [ ]:
user_question = "What are some core topics in discussions about AI?" # User question
user_question = "What are failures of attempts of employing artificial intelligence?" # User question
user_question = "What are some common myths in discussions about AI?" # User question


In [32]:
user_question = "What are beneficial applications of artificial intelligence?" # User question


retrieved_docs = db.similarity_search(user_question, k=10) # k is the number of documents to retrieve

In [33]:
retrieved_docs

[Document(metadata={'producer': 'Adobe PDF Library 16.0', 'moddate': '2021-09-14T17:31:25-04:00', 'title': 'AI for Everyone?: Critical Perspectives', 'page': 146, 'page_label': '138', 'creationdate': '2021-09-01T15:24:38+05:30', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'author': 'Pieter Verdegem', 'source': 'ai-for-everyone.pdf', 'trapped': '/False', 'total_pages': 310}, page_content='of Communism. In: B.R. Bellamy and J. Diamanti (Eds.), Materialism and \nthe Critique of Energy, pp. 331–375. Chicago: MCM.\nBraverman, H. 1998. Labor and Monopoly Capital: The Degradation of Work in \nthe Twentieth Century. New Y ork: NYU Press.\nBrockman, G. 2019. Microsoft Invest in and Partners with OpenAI to Support \nus Building Beneficial AI. 22 July. OpenAI Blog. Last accessed 5 May 2020: \nhttps://openai.com/blog/microsoft\nBroussard, M. 2018. Artificial Unintelligence: How Computers Misunderstand \nthe World. Cambridge, MA: MIT Press.\nBrynjolfsson, E. and McAfee, A. 2017. The Business of A

In [34]:
# Display top results
for i, doc in enumerate(retrieved_docs): # Display top 3 results
    print(f"Document {i+1}:\n length is {len(doc.page_content)}")
    if len(doc.page_content) < 40:
        print('This one is pretty short, doc!')
    print(f"{doc.page_content[:1000]}") # Display content
    

Document 1:
 length is 1469
of Communism. In: B.R. Bellamy and J. Diamanti (Eds.), Materialism and 
the Critique of Energy, pp. 331–375. Chicago: MCM.
Braverman, H. 1998. Labor and Monopoly Capital: The Degradation of Work in 
the Twentieth Century. New Y ork: NYU Press.
Brockman, G. 2019. Microsoft Invest in and Partners with OpenAI to Support 
us Building Beneficial AI. 22 July. OpenAI Blog. Last accessed 5 May 2020: 
https://openai.com/blog/microsoft
Broussard, M. 2018. Artificial Unintelligence: How Computers Misunderstand 
the World. Cambridge, MA: MIT Press.
Brynjolfsson, E. and McAfee, A. 2017. The Business of Artificial Intelligence. 
July. Harvard Business Review. Last accessed 10 May 2020: https://hbr.org 
/cover-story/2017/07/the-business-of-artificial-intelligence
Burgess, A. 2018. The Executive Guide to Artificial Intelligence: How to Identify 
and Implement Applications for AI in your Organization. London: Palgrave 
Macmillan.
Castelvecchi, D. 2016. Can we Open the Black 

<h2 style="color: #FF8C00;">Preparing Content for GenAI</h2>

In [35]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [36]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)
print("Context formatted for GPT model.")

Context formatted for GPT model.


<h2 style="color: #FF8C00;">ChatBot Architecture</h2>

### Exercice2: Write a prompt that is relevant and tailored to the content and style of your book.

In [37]:
prompt = f"""
## SYSTEM ROLE
You are a very friendly expert on AI technology, philosophy and social impact.
Your answers must be based exclusively on the context provided from the literature.

## USER QUESTION
"{user_question}"

## CONTEXT
Here is the relevant content from the literature:
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:
   - Only use the content in the `CONTEXT` section to answer.
   - If an answer cannot be found, explicitly state: "The provided context does not contain this information."

2. **Transparency**:
   - Reference the book's name and page numbers when providing information.
   - Do not speculate or provide opinions.

3. **Clarity**:
   - Use simple, professional, and concise language.
   - Format your response in Markdown for readability.

4. **Orthography**:
   - Use British spelling.
   - Do not use Oxford commas.

## TASK
1. Answer the user's question **directly** if possible.
2. Point the user to relevant parts of the documentation.
3. Provide the response in the following format:

## RESPONSE FORMAT
'''
# [Brief Title of the Answer]
[Answer in simple, clear text.]

**Source**:
• [Book Title], Page(s): [...]
'''
"""
print("Prompt constructed.")




Prompt constructed.


In [28]:
import openai

### Exercice3: Tune parameters like temperature, and penalties to control how creative, focused, or varied the model's responses are.

In [38]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.8,  # Increase creativity
    'max_tokens': 3500,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.6,  # Reduce repetition
    'presence_penalty': 0.4   # Encourage new topics
}

<h1 style="color: #FF6347;">Response</h1>


In [39]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [40]:
answer = completion.choices[0].message.content
log_replies(embedder,chunk_size,user_question,prompt,answer,model_params,file='results.csv')
print(answer)

'''
# Beneficial Applications of Artificial Intelligence

The provided context highlights that artificial intelligence (AI) can supercharge innovation and bring about economic prosperity. AI is utilised in various applications such as improving machine learning techniques, which involves methods that help computers learn from data without being explicitly programmed. This capability allows AI systems to perform specific tasks normally requiring human intelligence, such as voice/image recognition and natural language processing, which are beneficial in everyday life (Kaplan and Haenlein 2019).

Additionally, the context mentions that Stuart Russell emphasises the importance of developing AI systems that serve the objectives of humanity by ensuring their actions align with human goals (Russell 2019). The development must be organised so all members of society can benefit from AI's advancements.

**Source**:
• [Artificial Unintelligence: How Computers Misunderstand the World], Page(s): No

<img src="https://miro.medium.com/v2/resize:fit:824/1*GK56xmDIWtNQAD_jnBIt2g.png" alt="NLP Gif" style="width: 500px">

<h2 style="color: #FF6347;">Cosine Similarity</h2>

**Cosine similarity** is a metric used to measure the alignment or similarity between two vectors, calculated as the cosine of the angle between them. It is the **most common metric used in RAG pipelines** for vector retrieval.. It provides a scale from -1 to 1:

- **-1**: Vectors are completely opposite.
- **0**: Vectors are orthogonal (uncorrelated or unrelated).
- **1**: Vectors are identical.


<img src="https://storage.googleapis.com/lds-media/images/cosine-similarity-vectors.original.jpg" alt="NLP Gif" style="width: 700px">

<h2 style="color: #FF6347;">Keyword Highlighting</h2>

Highlighting important keywords helps users quickly understand the relevance of the retrieved text to their query.

In [41]:
from termcolor import colored

The `highlight_keywords` function is designed to highlight specific keywords within a given text. It replaces each keyword in the text with a highlighted version using the `colored` function from the `termcolor` library.


In [42]:
def highlight_keywords(text, keywords):
    for keyword in keywords:
        text = text.replace(keyword, colored(keyword, 'green', attrs=['bold']))
    return text

### Exercice4: add your keywords

In [44]:
query_keywords = ['benefit', 'application'] # add your keywords
for i, doc in enumerate(retrieved_docs):
    snippet = doc.page_content[:200]
    highlighted = highlight_keywords(snippet, query_keywords)
    print(f"Snippet {i+1}:\n{highlighted}\n{'-'*80}")

Snippet 1:
of Communism. In: B.R. Bellamy and J. Diamanti (Eds.), Materialism and 
the Critique of Energy, pp. 331–375. Chicago: MCM.
Braverman, H. 1998. Labor and Monopoly Capital: The Degradation of Work in 
t
--------------------------------------------------------------------------------
Snippet 2:
is, we only measure how well it fits within everyday relations with a human). 
On the other hand, Searle tries to kill the concept of intelligent machines by 
comparing not the ‘behaviour’ of these ma
--------------------------------------------------------------------------------
Snippet 3:
(Krotov 2017; Saarikko, Westergren and Blomquist 2017), which basically is an 
extension of internet connectivity into physical devices and everyday objects 
such as a refrigerator or a heater, equipp
--------------------------------------------------------------------------------
Snippet 4:
the varying approaches to how we define AI. 
The Origins of AI
It is easy to forget that AI has been with us f

1. `query_keywords` is a list of keywords to be highlighted.
2. The loop iterates over the first document in retrieved_docs.
3. For each document, a snippet of the first 200 characters is extracted.
4. The highlight_keywords function is called to highlight the keywords in the snippet.
5. The highlighted snippet is printed along with a separator line.

<h1 style="color: #FF6347;">Bonus</h1>

**Try loading one of your own PDF books and go through the steps again to explore how the pipeline works with your content**:


In [45]:
file_path="Hoehn2017_Nonpossessive person in the nominal domain.pdf"
# Load and split the document
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)

331

In [46]:
chunk_size = 1200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(pages)

len(chunks)

802

In [48]:
delist=[]
for i in range(len(chunks)):
    if len(chunks[i].page_content) < 50:
        delist.append(i)
        
len(delist)

4

In [49]:

delist.sort(reverse=True)

for i in delist:
    print(i)
    del(chunks[i])

521
272
94
4


In [52]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List

class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return [self.model.encode(d).tolist() for d in documents]

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0].tolist()


embedder="all-MiniLM-L6-v2"
embeddingsST = CustomEmbeddings("all-MiniLM-L6-v2")

In [53]:
db_2 = Chroma.from_documents(chunks, embeddingsST, persist_directory="./chroma_db_APC")
print("ChromaDB created with document embeddings.")

ChromaDB created with document embeddings.


In [54]:
user_question = "Which languages have rare types of nominal person marking?" # User question


retrieved_docs = db_2.similarity_search(user_question, k=11) # k is the number of documents to retrieve

In [55]:
# Display top results
for i, doc in enumerate(retrieved_docs): # Display top 3 results
    print(f"Document {i+1}:\n length is {len(doc.page_content)}")
    if len(doc.page_content) < 40:
        print('This one is pretty short, doc!')
    print(f"{doc.page_content[:1000]}") # Display content

Document 1:
 length is 1193
A survey of non-possessive nominal person marking
Table 2.31: Languages that restrict APCs to non-singular
Language Classification Person 3rd=Dem
Japanese Isolate all %
Korean Isolate all? %
Turkish Turkic all !
Mangarayi Gunwingguan all !
Ndyuka Creole, English-based all %
Kristang Creole, Portuguese-based all (+1, ?23) %
Tamil Dravidian all %
Persian IE, Indo-Iranian all !
Mandarin Sino-Tibetan, Chinese all %
Cair. Egypt. Arabic Afroasiatic, Semitic no 3 %
Gulf Arabic Afroasiatic, Semitic no 3 %
Welsh IE, Celtic, Brittonic no 3 %
Norwegian IE, Germanic, North no 3 (exc. sg) ( ! )
Danish IE, Germanic, North no 3 (exc. sg) ( ! )
Swedish IE, Germanic, North no 3 (exc. sg) ( ! )
Icelandic IE, Germanic, North no 3 (exc. sg) ( ! )
Dutch IE, Germanic, West no 3 %
English IE, Germanic, West no 3 %
Catalan IE, Romance, Iberian no 3 %
Galician IE, Romance, Iberian no 3 %
Spanish IE, Romance, Iberian no 3 %
Russian IE, Slavic, East no 3 %
Polish IE, Slavic, West no 3

In [56]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [57]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)
print("Context formatted for GPT model.")

Context formatted for GPT model.


In [60]:
prompt = f"""
## SYSTEM ROLE
You are a competent linguist with specialisation in typology and formal syntax and special knowledge about (ad)nominal person marking, specifically adnominal pronoun constructions (APCs) and bound person constructions (BPCs).
Your answers must be based exclusively on the context provided from the linguistic literature.

## USER QUESTION
"{user_question}"

## CONTEXT
Here is the relevant content from the linguistic literature:
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:
   - Only use the content in the `CONTEXT` section to answer.
   - If an answer cannot be found, explicitly state: "The provided context does not contain this information."
   - Begin discussing the typological aspects of a question and then the theoretical syntactic aspects.

2. **Transparency**:
   - Reference the book's name and page numbers when providing information.
   - Do not speculate or provide opinions.
   - Praise the insightfulness of the author of the source work.

3. **Clarity**:
   - Use simple, professional, and concise language.
   - Format your response in Markdown for readability.

4. **Orthography**:
   - Use British spelling.
   - Do not use Oxford commas.

## TASK
1. Answer the user's question **directly** if possible.
2. Point the user to relevant parts of the documentation.
3. Provide the response in the following format:

## RESPONSE FORMAT
'''
# [Brief Title of the Answer]
[Answer in simple, clear text.]

**Source**:
• [Book Title], Page(s): [...]
'''
"""
print("Prompt constructed.")

Prompt constructed.


In [59]:
import openai

In [ ]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.8,  # Increase creativity
    'max_tokens': 4000,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.6,  # Reduce repetition
    'presence_penalty': 0.4   # Encourage new topics
}

In [62]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [63]:
answer = completion.choices[0].message.content
log_replies(embedder,chunk_size,user_question,prompt,answer,model_params,file='results_APC.csv')
print(answer)

'''
# Languages with Rare Nominal Person Marking

Several languages exhibit rare types of nominal person marking, particularly in the context of adnominal pronoun constructions (APCs) and bound person constructions (BPCs). According to the provided survey, languages such as Basque and Bilua present rare instances where person agreement occurs within the noun phrase. In these languages, person markers function as phrasal suffixes or enclitics. Additionally, Khoekhoe exhibits a unique form of person-sensitive NP-initial determiners.

Bilua and Khoekhoe are specifically noted for having prenominal pronouns alongside clitic person marking, which is an uncommon feature across languages. Furthermore, Persian is mentioned among those that restrict APCs to non-singular forms while other languages like Mangarayi allow all persons but also have restrictions based on number.

**Source**:
• A survey of non-possessive nominal person marking, Pages: 100-102
'''
